# DEV - Curve Intersection

Some source code from `chiller_v09_t11.py`:

```python

import numpy as np

def _rect_inter_inner(self, x1, x2):
    n1 = x1.shape[0] - 1
    n2 = x2.shape[0] - 1
    X1 = np.c_[x1[:-1], x1[1:]]
    X2 = np.c_[x2[:-1], x2[1:]]
    S1 = np.tile(X1.min(axis=1), (n2, 1)).T
    S2 = np.tile(X2.max(axis=1), (n1, 1))
    S3 = np.tile(X1.max(axis=1), (n2, 1)).T
    S4 = np.tile(X2.min(axis=1), (n1, 1))
    return S1, S2, S3, S4

def _rectangle_intersection_(self, x1, y1, x2, y2):
    S1, S2, S3, S4 = self._rect_inter_inner(x1, x2)
    S5, S6, S7, S8 = self._rect_inter_inner(y1, y2)

    C1 = np.less_equal(S1, S2)
    C2 = np.greater_equal(S3, S4)
    C3 = np.less_equal(S5, S6)
    C4 = np.greater_equal(S7, S8)

    ii, jj = np.nonzero(C1 & C2 & C3 & C4)
    return ii, jj

def _intersection(self, x1, y1, x2, y2):
    x2 = np.asarray(x2)
    y2 = np.asarray(y2)
    ii, jj = self._rectangle_intersection_(x1, y1, x2, y2)
    n = len(ii)

    dxy1 = np.diff(np.c_[x1, y1], axis=0)
    dxy2 = np.diff(np.c_[x2, y2], axis=0)

    T = np.zeros((4, n))
    AA = np.zeros((4, 4, n))
    AA[0:2, 2, :] = -1
    AA[2:4, 3, :] = -1
    AA[0::2, 0, :] = dxy1[ii, :].T
    AA[1::2, 1, :] = dxy2[jj, :].T

    BB = np.zeros((4, n))
    BB[0, :] = -x1[ii].ravel()
    BB[1, :] = -x2[jj].ravel()
    BB[2, :] = -y1[ii].ravel()
    BB[3, :] = -y2[jj].ravel()

    for i in range(n):
        try:
            T[:, i] = np.linalg.solve(AA[:, :, i], BB[:, i])
        except:
            T[:, i] = np.NaN

    in_range = (T[0, :] >= 0) & (T[1, :] >= 0) & (T[0, :] <= 1) & (T[1, :] <= 1)

    xy0 = T[2:, in_range]
    xy0 = xy0.T
    return xy0[:, 0], xy0[:, 1]
def _fan_dp(self, n0100):
    if self.fan_type not in ["AC", "TC"]:
        self.fan.new_curve(n0100)
    else:
        self.fan.new_curve_tc(n0100)
    qa_min = self.fan.u_in[0]
    qa_max = self.fan.u_in[-1]
    u_fan = self.fan.u_in
    dp_fan = self.fan.dp_in
    while True:
        u_he, dp_he = self._pres_loss_air(qa_min, qa_max)
        qa_arr, dp_tot_arr = self._intersection(u_he, dp_he, u_fan, dp_fan)
        try:
            q_a = qa_arr[0]
            dp_tot = dp_tot_arr[0]
            break
        except:
            qa_min = 0
            u_fan.append(0)
            dp_fan.append(dp_fan[-1])
    return q_a, dp_tot
```